## import库

In [1]:
import os
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

import nltk
# nltk.download()
from nltk.corpus import stopwords

## 用pandas读入训练数据

In [2]:
datafile = os.path.join('..', 'data', 'labeledTrainData.tsv')
df = pd.read_csv(datafile, sep='\t', escapechar='\\')
print('Number of reviews: {}'.format(len(df)))
df.head()

Number of reviews: 25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
df['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

## 对影评数据做预处理，有以下缓解：
1. 去除html标签
2. 移除标点
3. 切成词/token
4. 去掉停用词
5. 重组为新的句子

In [4]:
def display(text, title):
    print(title)
    print('\n==============================\n')
    print(text)

In [5]:
raw_example = df['review'][0]
display(raw_example, '原始数据')

原始数据


With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it final

In [6]:
example = BeautifulSoup(raw_example, 'html.parser').get_text()
display(example, '去掉HTML标签的数据')

去掉HTML标签的数据


With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only

In [7]:
example_letters = re.sub(r'[^a-zA-Z]', ' ', example)
display(example_letters, '去掉标点的数据')

去掉标点的数据


With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on 

In [8]:
words = example_letters.lower().split()
display(words, '纯列表数据')

纯列表数据


['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again', 'maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', 'moonwalker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', 'some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'mj', 's', 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', 'm', 'kay', 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'all', '

In [9]:
# 下载停用词和其他语料会用到
# nltk.download()
# words_nostop = [w for w in words if w not in stopwords.words('english')]

In [10]:
# 使用存在的
stopwords = {}.fromkeys([line.rstrip() for line in open('../stopwords.txt')])
words_nostop = [w for w in words if w not in stopwords]
display(words_nostop, '去掉停用词数据')

去掉停用词数据


['stuff', 'moment', 'mj', 've', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'insight', 'guy', 'cool', 'eighties', 'mind', 'guilty', 'innocent', 'moonwalker', 'biography', 'feature', 'film', 'remember', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'press', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'michael', 'jackson', 'remotely', 'mj', 'hate', 'boring', 'call', 'mj', 'egotist', 'consenting', 'movie', 'mj', 'fans', 'fans', 'true', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 'mj', 'dead', 'bad', 'mj', 'overheard', 'plans', 'nah', 'joe', 'pesci', 'character', 'ranted', 'people', 'supplying', 'drugs', 'dunno', 'hates', 'mj', 'music', 'lots', 'cool', 'mj', 'car', 'robot', 'speed', 'demon', 'sequence', 'direc

In [11]:
eng_stopword = set(stopwords)
def clean_text(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'^a-zA-Z', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopword]
    return ' '.join(words)

In [12]:
clean_text(raw_example)

"stuff moment mj started listening music, watching odd documentary there, watched wiz watched moonwalker again. insight guy cool eighties mind guilty innocent. moonwalker biography, feature film remember cinema originally released. subtle messages mj's feeling press obvious message drugs bad m'kay.visually impressive michael jackson remotely mj hate boring. call mj egotist consenting movie mj fans fans true nice him.the actual feature film bit finally starts 20 minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord. mj dead bad me. mj overheard plans? nah, joe pesci's character ranted people supplying drugs dunno, hates mj's music.lots cool mj car robot speed demon sequence. also, director patience saint filming kiddy bad sequence directors hate kid bunch performing complex dance scene.bottom line, movie people mj level (which people). not, stay away. wholesome message ironically mj's bestest buddy movie girl! michael jackson talented people gra

## 清晰数据添加到dataframe里

In [13]:
df['clean_review'] = df.review.apply(clean_text)
df.head()

C:\Users\Alephant\.conda\envs\nlp\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,id,sentiment,review,clean_review
0,5814_8,1,With all this stuff going down at the moment w...,"stuff moment mj started listening music, watch..."
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin...","""the classic war worlds"" timothy hines enterta..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film starts manager (nicholas bell) investors ...
3,3630_4,0,It must be assumed that those who praised this...,"assumed praised film (""the filmed opera ever,""..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious 80's ...


## 抽取 bag of words特征（用sklearn的CountVectorizer）

In [14]:
vectorizer = CountVectorizer(max_features=5000)
train_data_features = vectorizer.fit_transform(df.clean_review).toarray()
train_data_features.shape

(25000, 5000)

In [15]:
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## 训练分类器

In [16]:
forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(train_data_features, df.sentiment)

## 在训练即上做个predict看看效果如何

In [17]:
confusion_matrix(df.sentiment, forest.predict(train_data_features))

array([[12500,     0],
       [    0, 12500]], dtype=int64)

## 删除不用的占内容变量

In [19]:
del df
del train_data_features

## 读取测试数据进行预测

In [21]:
datafile = os.path.join('..', 'data', 'testData.tsv')
df = pd.read_csv(datafile, sep='\t', escapechar='\\')
print('Number of reviews: {}'.format(len(df)))
df['clean_review'] = df.review.apply(clean_text)
df.head()

Number of reviews: 25000


C:\Users\Alephant\.conda\envs\nlp\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,id,review,clean_review
0,12311_10,"Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there...","naturally film main themes mortality, nostalgia, loss innocence surprising rated highly viewers ones. craftsmanship completeness film enjoy. pace steady constant, characters engaging, relationship..."
1,8348_2,"This movie is a disaster within a disaster film. It is full of great action scenes, which are only meaningful if you throw away all sense of reality. Let's see, word to the wise, lava burns you; s...","movie disaster disaster film. action scenes, meaningful throw sense reality. see, word wise, lava burns you; steam burns you. stand lava. diverting minor lava flow difficult, one. scares movie.eve..."
2,5828_4,"All in all, this is a movie for kids. We saw it tonight and my child loved it. At one point my kid's excitement was so great that sitting was impossible. However, I am a great fan of A.A. Milne's ...","all, movie kids. tonight child loved it. kid's excitement sitting impossible. however, fan a.a. milne's books subtle hide wry intelligence childlike quality leading characters. film subtle. shame ..."
3,7186_2,"Afraid of the Dark left me with the impression that several different screenplays were written, all too short for a feature length film, then spliced together clumsily into this Frankenstein's mon...","afraid dark left impression screenplays written, short feature length film, spliced clumsily frankenstein's monster.at best, protagonist, lucas, creepy. hard draw bead secondary characters, sympat..."
4,12128_7,"A very accurate depiction of small time mob life filmed in New Jersey. The story, characters and script are believable but the acting drops the ball. Still, it's worth watching, especially for the...","accurate depiction time mob life filmed jersey. story, characters script believable acting drops ball. still, worth watching, strong images, viewed 25 ago.a hood steps starts bigger (tries to) wro..."


In [22]:
test_data_features = vectorizer.transform(df.clean_review).toarray()
test_data_features.shape

(25000, 5000)

In [23]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'id':df.id, 'sentiment':result})

In [24]:
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,0
